In [1]:
import torch
import random

import torch.nn as nn

/Users/luan/Developer/bow-text-classifier/.venv/lib/python3.11/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


In [2]:
from bow_text_classifier.data import (
    get_dataset_filepath,
    _parse_data,
    _create_dict,
    _create_tensors,
)

# with open(get_dataset_filepath("dev"), "r") as f:
#     dev_data = _parse_data(f.read())

with open(get_dataset_filepath("train"), "r") as f:
    train_data = _parse_data(f.read())

with open(get_dataset_filepath("test"), "r") as f:
    test_data = _parse_data(f.read())


word_to_index, tag_to_index = _create_dict(train_data)

word_to_index, tag_to_index = _create_dict(
    test_data, word_to_index, tag_to_index, check_unk=True
)

number_of_words = len(word_to_index)
number_of_tags = len(tag_to_index)

train_data = list(_create_tensors(train_data, word_to_index, tag_to_index))
test_data = list(_create_tensors(test_data, word_to_index, tag_to_index))

number_of_words = len(word_to_index)
number_of_tags = len(tag_to_index)

In [3]:
from bow_text_classifier.nn import BoW, sentence_to_tensor

device = "cuda" if torch.cuda.is_available() else "cpu"


type = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor
out = sentence_to_tensor("i love dogs", word_to_index).type(type)
test_model = BoW(number_of_words, number_of_tags).to(device)

test_model(out)

tensor([[-0.0064,  0.0095,  0.0164, -0.0156,  0.0392]],
       grad_fn=<ViewBackward0>)

In [5]:
from bow_text_classifier.nn import train_bow

# train and test the BoW model
model = BoW(number_of_words, number_of_tags).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
type = torch.LongTensor

if torch.cuda.is_available():
    model.to(device)
    type = torch.cuda.LongTensor


# call the train_bow function
train_bow(model, optimizer, criterion, train_data, test_data, type)

TypeError: train_bow() takes 5 positional arguments but 6 were given